In [1]:
try:
    with open("../global_setup.py") as setupfile:
        exec(setupfile.read())
except FileNotFoundError:
    print('Setup already completed')

In [2]:
import pprint
import numpy as np
from gensim.models.fasttext import FastText
from scipy.spatial.distance import cdist
import re
# ESA relatedness package
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
from src.text.document_retrieval.wikipedia import Wikipedia # Generic Wikipedia class
wikipedia = Wikipedia(
    language="Danish",
    cache_directory_url=False
)

Loading parsed documents.
Loading preprocessed documents.
Wikipedia loaded.


In [4]:
class RsspediaInit:
    def __init__(self, wikipedia: Wikipedia, embedding_composition = "sum"):
        self.embedding_composition = embedding_composition
        self.search_results = []
        self.content = self.wikipedia_results = None
        # Initialize wikipedia
        self.wikipedia = wikipedia
        # Remove all the line breaks and caret returns from wiki texts
        pattern = re.compile('[\n\r ]+', re.UNICODE)
        self.texts = [self.wikipedia.documents[i].text for i in range(len(self.wikipedia.documents))]
        self.texts = [pattern.sub(' ', self.texts[i]) for i in range(len(self.texts))]
        self.texts_clean = [self.getCleanWordsList(self.texts[i], return_string = True) for i in range(len(self.texts))]
        
        # Calculate tf-idf representation for wiki texts (takes time)
        self._transformer = TfidfVectorizer(stop_words = None, norm = "l2", use_idf = True, sublinear_tf = False)
        self._Y = self._transformer.fit_transform(self.texts_clean)
        
        # Fasttext: initialize the model
        bin_path = Path("data", "fasttext", "wiki.da.bin")
        self.model = FastText.load_fasttext_format(str(bin_path))
        
        # Fasttext: Compute vectorized representation for all wikipedia articles (takes time)
        i = 0
        i_max = 0
        n_removed = 0
        self.wikipedia.documents_clean = self.wikipedia.documents.copy()
        self.wikipedia_abstract_vectors = []
        self.wikipedia_title_vectors = []
        pattern1 = re.compile('[^a-zA-Z0-9åÅøØæÆ]+', re.UNICODE)

        for n in range(len(self.wikipedia.documents)):
            # if abstract length is zero, remove it
            try:
                if len(pattern1.sub('', self.wikipedia.documents[n].abstract)) == 0:
                    del self.wikipedia.documents_clean[n - n_removed]
                    n_removed = n_removed + 1
                else:
                    self.wikipedia_abstract_vectors.append(self.sumVectorRepresentation(text = self.wikipedia.documents[n].abstract, type = self.embedding_composition))
                    self.wikipedia_title_vectors.append(self.sumVectorRepresentation(text = self.wikipedia.documents[n].title, type = self.embedding_composition))
                    
                    i = i + 1
                    if i_max > 0 and i > i_max:
                        break
            except IndexError as e:
                print("n: {}, n_removed: {}, w.d: {}, w.d_c: {}".format(n, n_removed, len(self.wikipedia.documents), len(self.wikipedia.documents_clean)))
    
    def getCleanWordsList(self, text, return_string = False):
        pattern = re.compile('[^a-zA-Z0-9åÅøØæÆ ]+', re.UNICODE)
        text = pattern.sub('', text)
        words = text.lower().strip().split()
        words_copy = words.copy()
        #stop_words = ["den", "det", "en", "et", "om", "for", "til", "at", "af", "på", "som", "og", 
        #              "jeg", "mig", "mine", "min", "mit", "du", "dig", "din", "dit", "dine", "han", "ham", "hun", "hende", 
        #              "de", "dem", "vi", "os", "sin", "sit", "sine", "sig"]
        
        stop_words = ["den", "det", "denne", "dette", "en", "et", "om", "for", "til", "at", "af", "på", "som", "og", "er", "i"]
        
        n_removed = 0
        for i in range(len(words)):
            if words[i] in stop_words:
                words_copy.pop(i - n_removed)
                n_removed = n_removed + 1
        if return_string:
            return ' '.join(words_copy)
        else:
            return words_copy
    
    def sumVectorRepresentation(self, text, verbose = False, type = "sum"):
        # Calculates vectorized represetnation of some text
        words = self.getCleanWordsList(text)
        text_vector = np.zeros(self.model.wv["a"].shape)
        if verbose:
            print("len: {}, words: {}".format(len(words), words))
        for i in range(len(words)):
            try:
                if type == "average":
                    text_vector = text_vector + self.model.wv[words[i]] / len(words)
                else: #sum
                    text_vector = text_vector + self.model.wv[words[i]]
            except KeyError as e:
                if verbose:
                    print("i: {}, e: {}".format(i, e))
                continue
        return text_vector


In [5]:
#rsspediainitAVG = RsspediaInit(wikipedia = wikipedia, embedding_composition = "average")
rsspediainit = RsspediaInit(wikipedia = wikipedia, embedding_composition = "sum")

In [16]:
class Rsspedia:
    def __init__(self, wikipedia: Wikipedia, rsspediainit: RsspediaInit):
        self.rsspediainit = rsspediainit
        self.embedding_composition = rsspediainit.embedding_composition
        self.search_results = []
        self.content = self.wikipedia_results = None
        # Initialize wikipedia
        self.wikipedia = wikipedia
        
        self.texts = rsspediainit.texts
        
        # Calculate tf-idf representation for wiki texts (takes time)
        self._transformer = rsspediainit._transformer
        self._Y = rsspediainit._Y
        
        # Fasttext: initialize the model
        self.model = rsspediainit.model
        
        #self.wikipedia.documents_clean 
        self.wikipedia_abstract_vectors = rsspediainit.wikipedia_abstract_vectors
        self.wikipedia_title_vectors = rsspediainit.wikipedia_title_vectors
    
    def get_ngrams(self, text):
        words = self.rsspediainit.getCleanWordsList(text)
        words_copy = words.copy()
        n_removed = 0
        

        for i in range(len(words_copy)):
            if i > 0:
                words_copy.append(words_copy[i - 1] + " " + words_copy[i])

        return words_copy
    
    def cdist_func(self, A, B):
        # Calculates cosine distance
        dists = cdist(A, B, 'cosine')
        return np.argmin(dists, axis=0), dists #np.min(dists, axis=0)

    def display_beautifully(self, titles, texts, urls):
        formatted_result_list = ["<ol>"]
        for i in range(len(titles)):
            formatted_result = "\n".join([
                "<li>",
                f"<p><a href=\"{urls[i]}\">{titles[i]}</a></p>",
                f"<p>{texts[i]}</p>",
                "</li>"
            ])
            formatted_result_list.append(formatted_result)
        formatted_result_list.append("</ol>")
        formatted_results = "\n".join(formatted_result_list)
        return formatted_results

    def search_wiki(self, search_texts, n_matches = 3, search_type = "okapibm25", remove_similar = False, verbose = False, p = 0.5, d = 0.1):
        n_mult_factor = 3 # factor to multiply n_matches with
        n_matches = n_matches * 3 # this is done to remove very similar values from the results and ensure we have enough to return
        titles = [] 
        texts = []
        urls = []
        scores = []
        
        # (1) Remove unnecessary symbols from the search text
        pattern = re.compile('[^a-zA-Z0-9åÅøØæÆ ]+', re.UNICODE)

        if search_texts:
            for i, text in enumerate(search_texts):
                # (1) Remove unnecessary symbols from the search text
                text = pattern.sub('', text)
                
                if search_type == "okapibm25":
                    wikipedia_results, search_terms = self.wikipedia.search(query = text, k_1 = 1.2, b = 0.75)
                    for index, score in wikipedia_results[:n_matches].items():
                        document = self.wikipedia.documents[index]
                        titles.append(document.title)
                        texts.append(document.abstract)
                        urls.append(document.url)
                        scores.append(score)
                elif search_type == "esa_relatedness":
                    y = self._transformer.transform([text])
                    D = np.array((self._Y * y.T).todense())
                    indices = np.argsort(-D, axis=0)
                    titles = [self.wikipedia.documents[index].title for index in indices[:n_matches, 0]]
                    texts = [self.wikipedia.documents[index].abstract for index in indices[:n_matches, 0]]
                    urls = [self.wikipedia.documents[index].url for index in indices[:n_matches, 0]]
                elif search_type == "fasttext_a":
                    # Calculate the vectorized representation
                    text_vector = self.rsspediainit.sumVectorRepresentation(text = text, type = self.embedding_composition)

                    cdist_result = self.cdist_func(self.wikipedia_abstract_vectors, [text_vector])
                    cdist_list = cdist_result[1] # List of all the cosine distances
                    cdist_list_sorted = np.sort(cdist_list, axis = 0) # Sorted list of cosine distances - to get top N matches

                    for i in range(n_matches):
                        result = np.where(cdist_list == cdist_list_sorted[i])
                        document = self.wikipedia.documents_clean[result[0][0]]
                        titles.append(document.title)
                        texts.append(document.abstract)
                        urls.append(document.url)
                        scores.append(cdist_list[result])
                elif search_type == "fasttext_b":
                    ngrams = self.get_ngrams(text)
                    r = []
                    for i in range(len(ngrams)):
                        cdist_result = self.cdist_func(self.wikipedia_title_vectors, [self.rsspediainit.sumVectorRepresentation(text = ngrams[i], type = self.embedding_composition)])
                        cdist_result2 = self.cdist_func([self.rsspediainit.sumVectorRepresentation(text = text, type = self.embedding_composition)], [self.rsspediainit.sumVectorRepresentation(text = ngrams[i], type = self.embedding_composition)])

                        cdist_list1 = cdist_result[1] # List of all the cosine distances
                        cdist_list2 = cdist_result2[1]
                        cdist_list = (cdist_list1 * p + cdist_list2 * (1 - p))
                        cdist_list_sorted = np.sort(cdist_list, axis = 0) # Sorted list of cosine distances - to get top N matches
                        
                        for j in range(5):
                            x = np.where(cdist_list == cdist_list_sorted[j])[0]
                            r.append( (x, cdist_list[x][0]))
                            if verbose:
                                print("{} {} {} {}".format(x, wikipedia.documents_clean[x[0]].title, cdist_list[x], ngrams[i]))

                    # When np.where returns multiple matches, we flatten them
                    r_copy = r.copy()
                    uniques = []
                    for i in range(len(r)-1, -1, -1):
                        if len(r[i][0]) > 1:
                            r_copy.pop(i)
                            for j in range(len(r[i][0])):
                                r_copy.append( (np.array([r[i][0][j]]), r[i][1]))

                    # Remove duplicate wikipedia pages. They occur because different n-grams can match the same pages
                    for i in range(len(r_copy)-1,-1,-1):
                        if r_copy[i][0] in uniques:
                            r_copy.pop(i)
                        else:
                            uniques.append(r_copy[i][0])
                    
                    r = r_copy
                    # Transform into list of tuples
                    r = [ (r[i][0][0], r[i][1][0]) for i in range(len(r))]
                    # Sort the list of tuples by cosine distance
                    r = sorted(r, key=lambda tup: tup[1])
                    
                    for i in range(len(r)):
                        document = self.wikipedia.documents_clean[r[i][0]]
                        titles.append(document.title)
                        #print("{} {}".format(document.title, r[i][1]))
                        texts.append(document.abstract)
                        urls.append(document.url)
                        scores.append(r[i][1])
            
            if remove_similar:
                # Removing too similar search results
                # Get vectors of the result titles
                title_result_vectors = [self.rsspediainit.sumVectorRepresentation(text = titles[i], type = self.embedding_composition) for i in range(len(titles))]
                titles_pruned = titles.copy()
                n_removed = 0
                ids_removed = []
                for i in range(len(titles)):
                    # Get cosine distances
                    cdist_result = self.cdist_func(title_result_vectors, [self.rsspediainit.sumVectorRepresentation(text = titles[i], type = self.embedding_composition)])[1]
                    # Sort cosine distances
                    cdist_result_sorted = np.sort(cdist_result, axis = 0)
                    rd = []
                    for j in range(len(titles) - i):
                        if i != j + i:
                            x = np.where(cdist_result == cdist_result_sorted[j + i])[0]
                            rd.append( (x, cdist_result[x][0]))
                            if cdist_result[x][0] < 0.10 and i + j not in ids_removed:
                                titles_pruned.pop(i + j - n_removed)
                                texts.pop(i + j - n_removed)
                                urls.pop(i + j - n_removed)
                                scores.pop(i + j - n_removed)
                                n_removed = n_removed + 1
                                ids_removed.append(i + j)
                                #print("removed: {}".format(i + j))
                            #print("{}-th title: {}, {}-th title: {}, dist: {}".format(i, titles[i], j + i, titles[j + i], cdist_result[x]))
                titles = titles_pruned[:int(n_matches / n_mult_factor)]
        return titles[:int(n_matches / n_mult_factor)], texts[:int(n_matches / n_mult_factor)], urls[:int(n_matches / n_mult_factor)], scores[:int(n_matches / n_mult_factor)]


#rsspedia = Rsspedia(wikipedia, rsspediainitAVG)
rsspedia = Rsspedia(wikipedia, rsspediainit)

In [19]:
#test_headline = "FN's verdensmål sættes i centrum på Folkemødet"
test_headline = "Google fyrer 13 chefer og 35 medarbejdere for sexchikane"
#test_headline = "Søren Hansen om den »nøgne sandhed«: Erdogan går på diplomatiske listefødder for at forbedre sit forhold til USA"
#test_headline = "Søren Hansen"
titles, texts, urls, scores = rsspedia.search_wiki(search_texts = [test_headline], n_matches = 10, search_type = "okapibm25")
pprint.pprint(titles)

titles, texts, urls, scores = rsspedia.search_wiki(search_texts = [test_headline], n_matches = 10, search_type = "esa_relatedness")
pprint.pprint(titles)

titles, texts, urls, scores = rsspedia.search_wiki(search_texts = [test_headline], n_matches = 10, search_type = "fasttext_a")
pprint.pprint(titles)

titles, texts, urls, scores = rsspedia.search_wiki(search_texts = [test_headline], n_matches = 10, search_type = "fasttext_b")
pprint.pprint(titles)

['Beredskabsstyrelsen Højskole',
 'Sexchikane',
 'Beredskabsstyrelsen Teknisk Skole',
 'Division (regnskab)',
 'Google Apps for Work',
 'Knol',
 'Civitas A/S',
 'Google Hangouts',
 'Google',
 'Google Search Console']
['Google Apps for Work',
 'Google',
 'Google Briller',
 'Google Books',
 'Whispering Voices',
 'Primærrute 13',
 'Gustav Fechner',
 'Google Video',
 'Globus',
 'Google Docs']
['Watch Medier',
 'Bladkompagniet',
 'Finansforbundet',
 'Associated Press',
 'Jobindex',
 'Skandiaweb',
 'DELACOUR',
 'WarnerMedia',
 'Dansk Journalistforbund',
 'Deloitte']
['Sexchikane',
 'Google',
 'Google+',
 'Google Hangouts',
 'Google (søgemaskine)',
 'Google Briller',
 'Medarbejder',
 'Servicemedarbejder',
 'Chikane',
 'Medarbejdende ægtefælle']


In [ ]:
titles, texts, urls, scores = rsspedia.search_wiki(search_texts = [test_headline], n_matches = 5, search_type = "fasttext_b", p = 0.4)
pprint.pprint(titles)

In [ ]:
#test_headline = "FN's verdensmål sættes i centrum på Folkemødet"
#test_headline = "Google fyrer 13 chefer og 35 medarbejdere for sexchikane"
#test_headline = "Søren Hansen om den »nøgne sandhed«: Erdogan går på diplomatiske listefødder for at forbedre sit forhold til USA"

p = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

test_headlines = ["Justin Bieber elsker burger",
                  "Søren Hansen sover godt"]

for i in range(len(p)):
    for j in range(len(test_headlines)):
        titles, texts, urls, scores = rsspedia.search_wiki(search_texts = [test_headlines[j]], n_matches = 15, search_type = "fasttext_b", p = p[i], verbose = False)
        print("*** ({}) {} ***".format(p[i], test_headlines[j]))
        [print("({}) {:.5f} {}".format(k + 1, scores[k], titles[k])) for k in range(len(titles))]
        print("\n")


In [ ]:
rsspedia.cdist_func([rsspedia.wikipedia_title_vectors[44919]], [rsspedia.sumVectorRepresentation("sørEN")])[1]

In [ ]:
test_headline = "FN's verdensmål sættes i centrum på Folkemødet"
#test_headline = "Google fyrer 13 chefer og 35 medarbejdere for sexchikane"
#test_headline = "Søren Hansen om den »nøgne sandhed«: Erdogan går på diplomatiske listefødder for at forbedre sit forhold til USA"

d = [0.01, 0.05, 0.1, 0.2]

test_headlines = ["Justin Bieber elsker burger",
                  "Søren Hansen sover godt"]

for i in range(len(d)):
    for j in range(len(test_headlines)):
        titles, texts, urls, scores = rsspedia.search_wiki(search_texts = [test_headlines[j]], n_matches = 15, search_type = "fasttext_b", d = d[i], verbose = False)
        print("*** ({}) {} ***".format(d[i], test_headlines[j]))
        [print("({}) {:.5f} {}".format(k + 1, scores[k], titles[k])) for k in range(len(titles))]
        print("\n")

In [8]:
rsspedia.wikipedia_abstract_vectors[99]

array([-4.28305321e+00,  2.55240115e+00, -1.45726916e+00, -8.23756914e+00,
       -8.64112043e+00, -6.47998441e+00, -1.29929687e+00, -9.55795763e-01,
        1.65495173e+01, -1.69481530e+01, -1.36593221e+01, -5.81666787e+00,
       -3.23517251e+00,  2.63173723e+01,  1.38813533e+01,  7.31259770e+00,
        1.57536662e+01,  2.09791069e+01, -5.90013766e+00, -3.35569414e+00,
        1.15768945e+01,  1.14082091e+01, -3.92751399e+00, -1.80111527e+01,
        7.23412335e+00, -4.99937767e+00, -1.94241644e+01, -1.12354711e+01,
       -1.51957655e+01, -1.76585835e+01, -1.45289562e+01,  1.16943159e+01,
       -7.11829895e+00,  5.23034242e+00, -8.89993020e+00,  1.65516643e-01,
        6.65601367e+00, -2.77061326e+00,  1.27500313e+01, -9.06576821e+00,
        1.39543985e+00,  2.43877220e+01, -9.57371298e+00,  6.56653776e+00,
       -8.08751804e+00, -9.48655071e+00,  1.14961673e+01, -9.75480168e+00,
       -1.77268107e+01, -5.30189526e+00,  2.77642972e-01,  4.23708503e+00,
        7.65727830e+00, -

In [13]:
rsspediainit.sumVectorRepresentation("bug")

array([ 0.05014786,  0.32823414, -0.2863861 , -0.14378658, -0.20674805,
        0.14268966,  0.25108913,  0.31807059, -0.45580789, -0.54190654,
       -0.4586145 ,  0.20334904, -0.12572755,  0.4007116 ,  0.02855139,
       -0.24636118,  0.08269521, -0.43348765, -0.09566914,  0.20010217,
       -0.14064902,  0.45069507, -0.09785812, -0.13032831,  0.5983457 ,
        0.16919152, -0.2981205 ,  0.60975236, -0.26399106, -0.08436299,
       -0.46794125,  0.31169823,  0.58768255,  0.02459755,  0.0424697 ,
        0.40876111,  0.00973499, -0.29088885,  0.60246372, -0.33718371,
       -0.12872784,  1.03731573, -0.47151724, -0.37271887, -0.48083046,
       -0.70766419, -0.14369157, -0.10628562,  0.15250066, -0.73531884,
       -0.28511685, -0.5384478 ,  0.22355257, -0.28713101, -0.44965044,
       -0.60707319,  0.11590097, -0.1102048 , -0.0708666 , -0.09127642,
       -0.14107014,  0.166862  , -0.01928574,  0.06991053, -0.04413012,
        0.10818345, -0.03567166, -0.02626268,  0.23833179, -0.41